<a href="https://colab.research.google.com/github/lucaCosca/JetNet_ML-Final-Project/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet-34 Architechture


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

import gdown

### Loading The Data

In [4]:
url1 = "https://drive.google.com/uc?id=1P4SYbafssKz-6oPXjqTD7aBiCawQjHEk"
output1 = "Y_N70000_lpx50_gqtwz.npz"

url2 = "https://drive.google.com/uc?id=1S_QInod3zyO8Vj-YGQfuM1eVPnwiIeRX"
output2 = "X_N70000_lpx50_gqtwz.npz"

gdown.download(url1, output1, quiet=False)
gdown.download(url2, output2, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1P4SYbafssKz-6oPXjqTD7aBiCawQjHEk
To: /content/Y_N70000_lpx50_gqtwz.npz
100%|██████████| 1.12M/1.12M [00:00<00:00, 59.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1S_QInod3zyO8Vj-YGQfuM1eVPnwiIeRX
From (redirected): https://drive.google.com/uc?id=1S_QInod3zyO8Vj-YGQfuM1eVPnwiIeRX&confirm=t&uuid=0b73b943-ccfa-4e59-b5d0-f0ca3bf1ea84
To: /content/X_N70000_lpx50_gqtwz.npz
100%|██████████| 1.40G/1.40G [00:27<00:00, 50.2MB/s]


'X_N70000_lpx50_gqtwz.npz'

In [5]:
X_Data = np.load("X_N70000_lpx50_gqtwz.npz")["arr_0"]
Y_Data = np.load("Y_N70000_lpx50_gqtwz.npz")["arr_0"][:,0] # took only the feature about the class of the particle, it cuts the mass

### Data Preprocessing

In [6]:
# data split
num_images = X_Data.shape[0]

train_end = int(0.6 * num_images)
test_end = train_end + int(0.2 * num_images)

X_Data = X_Data.reshape(num_images, 50, 50, 1)

X_train, X_test, X_val = X_Data[:train_end], X_Data[train_end:test_end], X_Data[test_end:]
Y_train, Y_test, Y_val = Y_Data[:train_end], Y_Data[train_end:test_end], Y_Data[test_end:]

from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False)
Y_train_onehot = one_hot_encoder.fit_transform(Y_train.reshape((-1,1)))
Y_valid_onehot = one_hot_encoder.fit_transform(Y_val.reshape((-1,1)))
Y_test_onehot = one_hot_encoder.fit_transform(Y_test.reshape((-1,1)))

# The data is alredy normalized


### Network Architecture


In [7]:
# Residual Block Layer definition

class ResidualBlock(tf.keras.layers.Layer):

  def __init__(self, filters, strides=1, **kwargs):
    super().__init__(**kwargs)

    self.main_layers = [
        tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=strides, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same'),
        tf.keras.layers.BatchNormalization()
    ]

    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
          tf.keras.layers.Conv2D(filters=filters, kernel_size=1, strides=strides, padding='same'),
          tf.keras.layers.BatchNormalization()
      ]

  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)

    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)

    return tf.keras.layers.Activation('relu')(Z + skip_Z)

In [ ]:
ResNet34 = tf.keras.Sequential([

    tf.keras.layers.Input(shape=[50, 50, 1]),
    tf.keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(),

    ResidualBlock(filters=64),
    ResidualBlock(filters=64),
    ResidualBlock(filters=64),

    ResidualBlock(filters=128, strides=2),
    ResidualBlock(filters=128),
    ResidualBlock(filters=128),
    ResidualBlock(filters=128),

    ResidualBlock(filters=256, strides=2),
    ResidualBlock(filters=256),
    ResidualBlock(filters=256),
    ResidualBlock(filters=256),
    ResidualBlock(filters=256),
    ResidualBlock(filters=256),

    ResidualBlock(filters=512, strides=2),
    ResidualBlock(filters=512),
    ResidualBlock(filters=512),

    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5, activation='softmax')
])